## Chap9. GAN

### 서로 대립하는 두 신경망을 경쟁시켜가며 결과물 생성 방법을 학습

### GAN 기본 구조
: 실제 이미지 -> 구분자 / 노이즈 -> 생성자(임의의 이미지를 만든다) -> 구분자를 통해 판단하게함(진짜인지 아닌지) \
=> 생성자는 구분자를 속여 진짜처럼 보이게 하고, 구분자는 생성자가 만든 이미지를 최대한 가짜라고 구분하도록 훈련 

 * 구분자에 넣을 이미지 2개 (실제, 생성된 가짜) 
 * 가짜 이미지는 노이즈에서 생성(노이즈를 입력하여 생성)

### 활용방안 
 * 사진을 고흐 풍 그림으로 그려줌
 * 선으로만 그려진 만화를 자동 채색
 * 모자이크를 없앰 
 * 이미지 생성에 주로 쓰이나 자연어 문장 생성에도 활발하게 연구가 진행됨



### GAN 모델을 이용하여 MNIST 손글씨 숫자를 무작위로 생성 + 원하는 숫자에 해당하는 이미지 생성 모델 구현 

In [4]:
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
#하이퍼파라미터 설정 

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
# 히든 레이어의 뉴런 갯수
n_hidden = 256  
# 입력값 크기(이미지 픽셀수)
n_input = 28*28  
#생성자의 입력값(노이즈 크기)
n_noise = 128

In [6]:
# 1.신경망 모델 구성 

## 플레이스홀더: 비지도 학습이므로 x 하나와 노이즈 입력할 z 

X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [8]:
# 생성자 신경망 변수 설정

## 은닉층으로 출력하기 위한 변수 
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
## 출력층에 사용 할 변수 
# 가중치의 변수 크기 == 실제 이미지의 크기(=n_input)
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [9]:
# 구분자 신경망 변수 설정 

## 진짜와 얼마나 가까운가 판단 : 0-1 사이 값을 출력 
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden,1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

* 실제 이미지 판별 구분자 신경망과 생성한 이미지 판별 구분자 신경망은 같은 변수를 사용해야함.


=>같은 신경망으로 구분을 시켜야 진짜 이미지와 가짜 이미지를 구분하는 특징들을 동시에 잡아낼 수 있음. 

In [10]:
# 생성자 신경망 구성 
## 무작위로 생성한 노이즈를 받아 가중치, 편향 반영하여 은닉층 만들고,
## 은닉층에서 실 이미지와 같은 크기의 결괏값을 출력함. 
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1)+G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden,G_W2)+G_b2)
    return output

# 구분자 신경망 
## 0~1 사이의 스칼라값 출력 : 활성화함수(sigmoid)

def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1)+D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden,D_W2)+D_b2)
    return output

# 무작위 노이즈 생성 유틸리티 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size,  n_noise))
# Z: 노이즈 , G: 생성자, X: 진짜 이미지 
#노이즈를 이용, 랜덤 이미지 생성
G=generator(Z)
# 노이즈로 생성한 이미지가 진짜이미지인지 판별한 값을 구함
D_gene = discriminator(G)
#진짜 이미지를 이용해 판별한 값을 구함 
D_real = discriminator(X)

In [12]:
# 손실값 구하기 
## 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록하는 손실값(경찰 학습용)
## 진짜라고 판단하도록 하는 손실값(위조지폐범 학습용)
## 진짜 이미지 판별값: D_real 가짜 이미지 판별값: D_gene

#경찰학습
loss_D = tf.reduce_mean(tf.log(D_real)+ tf.log(1-D_gene))

#위조지폐범 학습
#D_gene을 1에 가깝게 : 가짜이미지를 넣어도 진짜같다고 판별해야함
loss_G = tf.reduce_mean(tf.log(D_gene))

#### => 즉, GAN의 학습은 loss_D와 loss_G 모두를 최대화하는 것이다. 
: loss_G와 loss_D는 서로 경쟁관계이다


In [13]:
#loss_D를 구할 때는 구분자 신경망에 사용되는 변수들만 사용
#loss_G를 구할 때는 생성자 신경망에 사용되는 변수들만 사용하여 최적화 

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                        var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                        var_list=G_var_list)

In [15]:
# 2.신경망 모델 학습 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # 판별기와 생성기 신경망 각각 학습 
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    
# 3, 이미지 생성 및 저장
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: -0.4355 G loss: -2.141
Epoch: 0001 D loss: -0.1254 G loss: -2.996
Epoch: 0002 D loss: -0.1563 G loss: -2.944
Epoch: 0003 D loss: -0.5012 G loss: -1.617
Epoch: 0004 D loss: -0.4419 G loss: -1.498
Epoch: 0005 D loss: -0.2972 G loss: -2.516
Epoch: 0006 D loss: -0.3435 G loss: -2.848
Epoch: 0007 D loss: -0.2052 G loss: -2.853
Epoch: 0008 D loss: -0.2729 G loss: -2.5
Epoch: 0009 D loss: -0.256 G loss: -2.85
Epoch: 0010 D loss: -0.2965 G loss: -2.399
Epoch: 0011 D loss: -0.4296 G loss: -2.248
Epoch: 0012 D loss: -0.5139 G loss: -2.078
Epoch: 0013 D loss: -0.3999 G loss: -2.218
Epoch: 0014 D loss: -0.3435 G loss: -2.846
Epoch: 0015 D loss: -0.2906 G loss: -2.855
Epoch: 0016 D loss: -0.3752 G loss: -2.216
Epoch: 0017 D loss: -0.4065 G loss: -2.358
Epoch: 0018 D loss: -0.2976 G loss: -2.534
Epoch: 0019 D loss: -0.4735 G loss: -2.766
Epoch: 0020 D loss: -0.3916 G loss: -2.657
Epoch: 0021 D loss: -0.3916 G loss: -2.474
Epoch: 0022 D loss: -0.3596 G loss: -2.68
Epoch: 0023 D lo

### 무작위 숫자 생성이 아닌 원하는 숫자를 지정해 생성하는 모델

In [16]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈와 실제 이미지에, 각각에 해당하는 숫자에 대한 힌트를 넣어주기 위해 사용
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [17]:
# tf.layers 사용 
# 변수를 선언하지 않고 tf.variable_scope을 이용해 스코프 지정 가능 

#생성자 신경망

def generator(noise, labels):
    with tf.variable_scope('generator'):
        # tf.concat: noise 값에 labels 정보 추가
        inputs = tf.concat([noise, labels], 1)
        
        #tf.layers.dense : 은닉층과 출력층 구성 
        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,
                                 activation=tf.nn.sigmoid)

    return output

# 구분자 신경망 : 진짜 이미지를 판별할 때와 가짜 이미지를 판별할 때 똑같은 변수를 사용함
## scope.reuse_variables 함수를 이용해 이전 변수 재사용 
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        
        if reuse:
            scope.reuse_variables()

        inputs = tf.concat([inputs, labels], 1)

        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        #손실값 계산에 sigmoid_cross_entropy_with_logits 함수를 사용하기위해 출력값에 활성화 함수사용 X
        output = tf.layers.dense(hidden, 1,
                                 activation=None)

    return output

#노이즈 생성함수 : 균등분포 
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])


G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

#손실함수 
# 진짜 이미지를 판별하는 D_real 값은 1에 가깝게, 가짜 이미지를 판별하는 D_gene 값은 0에 가깝게

loss_D_real = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.zeros_like(D_gene)))

loss_D = loss_D_real + loss_D_gene
# 가짜 이미지를 진짜에 가깝게 만들도록 생성망을 학습시키기 위해, D_gene 을 최대한 1에 가깝도록 만든다
loss_G = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.ones_like(D_gene)))

vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                            var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                            var_list=vars_G)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# 신경망 모델 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Y: batch_ys, Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,
                           feed_dict={Y: mnist.test.labels[:sample_size],
                                      Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: 0.008958 G loss: 8.813
Epoch: 0001 D loss: 0.005673 G loss: 7.992
Epoch: 0002 D loss: 0.00631 G loss: 7.5
Epoch: 0003 D loss: 0.01362 G loss: 8.58
Epoch: 0004 D loss: 0.05124 G loss: 10.99
Epoch: 0005 D loss: 0.02676 G loss: 7.37
Epoch: 0006 D loss: 0.003319 G loss: 9.339
Epoch: 0007 D loss: 0.02108 G loss: 8.303
Epoch: 0008 D loss: 0.06314 G loss: 11.54
Epoch: 0009 D loss: 0.04215 G loss: 10.59
Epoch: 0010 D loss: 0.01539 G loss: 9.873
Epoch: 0011 D loss: 0.07194 G loss: 8.086
Epoch: 0012 D loss: 0.08398 G loss: 6.675
Epoch: 0013 D loss: 0.2344 G loss: 5.325
Epoch: 0014 D loss: 0.357 G loss: 5.668
Epoch: 0015 D loss: 0.4653 G loss: 4.671
Epoch: 0016 D loss: 0.2756 G loss: 4.128
Epoch: 0017 D loss: 0.4175 G loss: 3.659
Epoch: 0018 D loss: 0.2626 G loss: 3.922
Epoch: 0019 D loss: 0.507 G loss: 3.912
Epoch: 0020 D loss: 0.4913 G loss: 3.842
Epoch: 0021 D loss: 0.5308 G loss: 3.328
Epoch: 0022 D loss: 0.4366 G loss: 3.875
Epoch: 0023 D loss: 0.4958 G loss: 3.04
Epoch: 